#Building an image retrieval system with deep features


#Fire up GraphLab Create

In [1]:
import graphlab

#Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [2]:
image_train = graphlab.SFrame('image_train_data/')

2016-04-18 13:18:50,094 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: C:\Users\ADMINI~1\AppData\Local\Temp\graphlab_server_1460956725.log.0


This non-commercial license of GraphLab Create is assigned to liujunilmm@126.com and will expire on March 16, 2017. For commercial licensing options, visit https://dato.com/buy/.


#Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [3]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [4]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


#Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [5]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

#Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [6]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [7]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 195.011ms    |

| Done         |         | 100         | 342.019ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


We are going to create a simple function to view the nearest neighbors to save typing:

In [8]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [9]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 69.004ms     |

| Done         |         | 100         | 250.014ms    |

+--------------+---------+-------------+--------------+

In [10]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

##Finding similar images to a car

In [11]:
car = image_train[8:9]
car['image'].show()

In [12]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 54.003ms     |

| Done         |         | 100         | 224.013ms    |

+--------------+---------+-------------+--------------+

#Just for fun, let's create a lambda to find and show nearest neighbor images

In [13]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [14]:
show_neighbors(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 43.002ms     |

| Done         |         | 100         | 235.013ms    |

+--------------+---------+-------------+--------------+

In [15]:
show_neighbors(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 41.003ms     |

| Done         |         | 100         | 206.012ms    |

+--------------+---------+-------------+--------------+

# 1.Computing summary statistics of the data
Using the training data, compute the sketch summary of the ‘label’ column and interpret the results.

In [16]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


 So the least common catagory in training dataset is bird

#2. Creating category-specific image retrieval models

Split the SFrame with the training data into 4 different SFrames.

In [17]:
dog_data = image_train[image_train['label'] == 'dog'] 

In [18]:
cat_data = image_train[image_train['label'] == 'cat']

In [19]:
bird_data = image_train[image_train['label'] == 'bird']

In [20]:
automobile_data = image_train[image_train['label'] == 'automobile']

In [21]:
dog_model = graphlab.nearest_neighbors.create(dog_data,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [22]:
cat_model = graphlab.nearest_neighbors.create(cat_data,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [23]:
bird_model = graphlab.nearest_neighbors.create(bird_data,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [24]:
automobile_model = graphlab.nearest_neighbors.create(automobile_data,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [25]:
image_test = graphlab.SFrame('image_test_data/')

In [26]:
image_test[0:1]['image'].show()

In [75]:
dog_model.query(image_test[0:1])
##cat_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 11.001ms     |

| Done         |         | 100         | 61.004ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16976,37.4642628784,1
0,13387,37.5666832169,2
0,35867,37.6047267079,3
0,44603,37.7065585153,4
0,6094,38.5113254907,5


In [28]:
dog_neighbors = get_images_from_ids(dog_model.query(image_test[0:1]))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 11.001ms     |

| Done         |         | 100         | 80.005ms     |

+--------------+---------+-------------+--------------+

In [29]:
dog_neighbors['image'].show()

In [67]:
get_images_from_ids(cat_model.query(image_test[0:1]))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 11.001ms     |

| Done         |         | 100         | 63.004ms     |

+--------------+---------+-------------+--------------+

## Computing nearest neighbors accuracy using SFrame operations

In [32]:
image_test_cat = image_test[image_test['label'] == 'cat']

In [33]:
image_test_dog = image_test[image_test['label'] == 'dog']

In [34]:
image_test_bird = image_test[image_test['label'] == 'bird']

In [35]:
image_test_automobile = image_test[image_test['label'] == 'automobile']

In [43]:
dog_dog_neighbors = dog_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 7668

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 64000   | 12.5737     | 367.021ms    |

| Done         | 509000  | 100         | 474.028ms    |

+--------------+---------+-------------+--------------+

In [44]:
get_images_from_ids(dog_dog_neighbors)['image'].show()

In [41]:
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 7668

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 64000   | 12.5737     | 393.022ms    |

| Done         | 509000  | 100         | 458.026ms    |

+--------------+---------+-------------+--------------+

In [70]:
get_images_from_ids(dog_cat_neighbors)['image'].show()

In [45]:
dog_bird_neighbors = bird_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 7668

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 60000   | 12.5523     | 395.023ms    |

| Done         | 478000  | 100         | 445.026ms    |

+--------------+---------+-------------+--------------+

In [46]:
get_images_from_ids(dog_bird_neighbors)['image'].show()

In [47]:
dog_automobile_neighbors = automobile_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 7668

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 64000   | 12.5737     | 439.025ms    |

+--------------+---------+-------------+--------------+

In [48]:
get_images_from_ids(dog_automobile_neighbors)['image'].show()

In [51]:
dog_distances = graphlab.SFrame({'dog-automobile': dog_automobile_neighbors['distance'],'dog-bird': dog_automobile_neighbors['distance'],
                             'dog-cat':dog_cat_neighbors['distance'],'dog-dog':dog_dog_neighbors['distance']})

In [52]:
dog_distances

dog-automobile,dog-bird,dog-cat,dog-dog
41.9579761457,41.9579761457,36.4196077068,33.4773590373
46.0021331807,46.0021331807,38.8353268874,32.8458495684
42.9462290692,42.9462290692,36.9763410854,35.0397073189
41.6866060048,41.6866060048,34.5750072914,33.9010327697
39.2269664935,39.2269664935,34.778824791,37.4849250909
40.5845117698,40.5845117698,35.1171578292,34.945165344
45.1067352961,45.1067352961,40.6095830913,39.0957278345
41.3221140974,41.3221140974,39.9036867306,37.7696131032
41.8244654995,41.8244654995,38.0674700168,35.1089144603
45.4976929401,45.4976929401,42.7258732951,43.2422832585


In [62]:
def is_dog_correct(row):
    if dog_distances[row]['dog-dog'] < dog_distances[row]['dog-bird'] and dog_distances[row]['dog-automobile'] and dog_distances[0:1]['dog-cat']:
        i = 1
    else:
        i = 0
    return i

In [63]:
dog_distances.apply(is_dog_correct).sum()

1000L

Accuracy of predicting dog in the test data

In [64]:
dog_distances.apply(is_dog_correct).sum() / len(dog_distances)

1L